This corpus contains a large metadata-rich collection of fictional conversations extracted from raw movie scripts:



- 220,579 conversational exchanges between 10,292 pairs of movie characters

- involves 9,035 characters from 617 movies

- in total 304,713 utterances

- movie metadata included:

    - genres

    - release year

    - IMDB rating

    - number of IMDB votes

    - IMDB rating

- character metadata included:

    - gender (for 3,774 characters)

    - position on movie credits (3,321 characters)

- see README.txt (included) for details

https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

1.가상환경 설치

conda  create -n venv_chatbot python=3.5 anaconda

2.가상환경 실행
conda activate venv_chatbot

3.가상환경 종료
conda deactivate



Tensorfow 1.0.0 설치
Pip install tensorflow==1.0.0


In [3]:
import numpy as np
#import pandas as df
#import tensorflow as tf
import re

In [39]:
pwd

'C:\\Users\\cacki\\Projects\\chatbot_seq2seq-main'

In [40]:
ls

 C 드라이브의 볼륨: Windows
 볼륨 일련 번호: 1A8B-B9C0

 C:\Users\cacki\Projects\chatbot_seq2seq-main 디렉터리

2021-01-08  오후 04:11    <DIR>          .
2021-01-08  오후 04:11    <DIR>          ..
2021-01-08  오후 12:25    <DIR>          .ipynb_checkpoints
2021-01-08  오후 04:11         2,805,130 chatbot_.ipynb
2010-12-17  오후 03:20         6,760,930 movie_conversations.txt
2010-12-17  오후 03:20        34,641,919 movie_lines.txt
2021-01-07  오후 10:13               845 README.md
               4개 파일          44,208,824 바이트
               3개 디렉터리  563,456,421,888 바이트 남음


In [41]:
lines = open('movie_lines.txt', encoding='utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding='utf-8', errors = 'ignore').read().split('\n')

In [42]:
lines[:5] #대화 데이터 확인

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]

In [43]:
len(lines)

304714

In [44]:
conversations[:10] #대화구성 확인

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [9]:
conversations[:-1] #맨 뒷부분 아이템 확인

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

In [47]:
#creating a dictionary. each line and id
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ') #한 줄을 불러와서 +++$+++ 로 분리
    if len(_line) == 5:
        id2line[_line[0]] = _line[4] #'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!' 4번째가 대화로 이것을 딕셔너리에 저장
        

In [48]:
id2line # 전처리 완료  key, value로 구성

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

In [12]:
# 대화 구성 전처리
conversations_ids = []
for conversation in conversations[:-1]:
    # "u5 +++$+++ u8 +++$+++ m0 +++$+++ ['L103', 'L104', 'L105', 'L106', 'L107']" 에서 대화내용만 추출
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ", "")
    conversations_ids.append(_conversation.split(',')) # , 로 구분해놓는다.

In [13]:
conversations_ids[:5] #전처리 확인

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208']]

In [14]:
#이제 질문과 답변을 구분!
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]]) # 대화구성 내용을 한줄씩 가져와서 각 대화를 id2line에서 찾아서 첫번째것이 질문이고
        answers.append(id2line[conversation[i+1]]) # 두 번째 것이 답변. 이것을 각 questions, answers에 넣음

In [49]:
questions[:3] #check for questions 잘됨

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.']

In [50]:
answers[:3]#check for answers 잘됨

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"]

In [17]:
# 이러한 questions, answers는 학습시키는데 에러가 발생함, 그래서 조금 더 다음어야함 cleaning!
def clean_text(text):
    text = text.lower()# 소문자로 전환
    text = re.sub(r"i'm", "i am", text) #약어는 풀어서 표현
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"/'ll", "will", text)
    text = re.sub(r"/'ve", "have", text)
    text = re.sub(r"/'re", "are", text)
    text = re.sub(r"/'d", "would", text)
    text = re.sub(r"/'won't", "will not", text)
    text = re.sub(r"/'can't", "can not", text)
    text = re.sub(r"/[-()\"#/;:<>{}+=-~|.?,]", "", text) # 가장중요하고 일반적으로 사용하는 범용코드
    return text

In [18]:
#cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question)) # 질문을 가져와서 함수로 클리닝 하고 리스트로 저장함
    
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [19]:
clean_questions[:5] # 크리닝 잘됨

['can we make this quick?  roxanne korrine and andrew barrett are having an incredibly horrendous public break- up on the quad.  again.',
 "well, i thought we'd start with pronunciation, if that is okay with you.",
 'not the hacking and gagging and spitting part.  please.',
 "you're asking me out.  that is so cute. what's your name again?",
 "no, no, it's my fault -- we didn't have a proper introduction ---"]

In [20]:
clean_answers[:5] # 크리닝 잘됨

["well, i thought we'd start with pronunciation, if that is okay with you.",
 'not the hacking and gagging and spitting part.  please.',
 "okay... then how 'bout we try out some french cuisine.  saturday?  night?",
 'forget it.',
 'cameron.']

In [21]:
# Create Dictionary 각 단어들이 얼마나 등장하는지 사전에 담기
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [22]:
word2count #questions, answers에 포함된 모든 단어의 빈도수 계산완료!

{'can': 15281,
 'we': 27766,
 'make': 6558,
 'this': 27776,
 'quick?': 8,
 'roxanne': 1,
 'korrine': 1,
 'and': 63825,
 'andrew': 41,
 'barrett': 9,
 'are': 21626,
 'having': 1185,
 'an': 9447,
 'incredibly': 56,
 'horrendous': 1,
 'public': 285,
 'break-': 9,
 'up': 11139,
 'on': 23128,
 'the': 139853,
 'quad.': 1,
 'again.': 1558,
 'well,': 9011,
 'i': 179698,
 'thought': 4140,
 "we'd": 639,
 'start': 1381,
 'with': 24172,
 'pronunciation,': 2,
 'if': 18738,
 'that': 55059,
 'is': 68693,
 'okay': 480,
 'you.': 12686,
 'not': 25495,
 'hacking': 14,
 'gagging': 7,
 'spitting': 15,
 'part.': 161,
 'please.': 918,
 "you're": 19487,
 'asking': 608,
 'me': 25301,
 'out.': 2895,
 'so': 15409,
 'cute.': 97,
 "what's": 6778,
 'your': 29796,
 'name': 1701,
 'again?': 451,
 'no,': 7075,
 "it's": 25550,
 'my': 29391,
 'fault': 130,
 '--': 27441,
 "didn't": 8166,
 'have': 30185,
 'a': 101491,
 'proper': 107,
 'introduction': 9,
 '---': 120,
 'cameron.': 14,
 'thing': 3549,
 'is,': 1168,
 'cameron

In [23]:
word2count.items() #딕셔너리의 아이템을 확인한다.

dict_items([('can', 15281), ('we', 27766), ('make', 6558), ('this', 27776), ('quick?', 8), ('roxanne', 1), ('korrine', 1), ('and', 63825), ('andrew', 41), ('barrett', 9), ('are', 21626), ('having', 1185), ('an', 9447), ('incredibly', 56), ('horrendous', 1), ('public', 285), ('break-', 9), ('up', 11139), ('on', 23128), ('the', 139853), ('quad.', 1), ('again.', 1558), ('well,', 9011), ('i', 179698), ('thought', 4140), ("we'd", 639), ('start', 1381), ('with', 24172), ('pronunciation,', 2), ('if', 18738), ('that', 55059), ('is', 68693), ('okay', 480), ('you.', 12686), ('not', 25495), ('hacking', 14), ('gagging', 7), ('spitting', 15), ('part.', 161), ('please.', 918), ("you're", 19487), ('asking', 608), ('me', 25301), ('out.', 2895), ('so', 15409), ('cute.', 97), ("what's", 6778), ('your', 29796), ('name', 1701), ('again?', 451), ('no,', 7075), ("it's", 25550), ('my', 29391), ('fault', 130), ('--', 27441), ("didn't", 8166), ('have', 30185), ('a', 101491), ('proper', 107), ('introduction', 9

In [24]:
#두개의 딕셔너리에 질문, 답변의 단어를 유일한 숫자로 변환하여 제작
threshold = 20 # 보통 20번 이상 나오는 단어는 중요한 단어니까. 이것을 기준으로 각 단어들을 유일한 넘버를 매겨서 딕셔너리로 만들자. 20은 임의 기준값

questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1
        
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1

In [25]:
questionswords2int #단어를 유일한 숫자로 변환 성공

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'and': 4,
 'andrew': 5,
 'are': 6,
 'having': 7,
 'an': 8,
 'incredibly': 9,
 'public': 10,
 'up': 11,
 'on': 12,
 'the': 13,
 'again.': 14,
 'well,': 15,
 'i': 16,
 'thought': 17,
 "we'd": 18,
 'start': 19,
 'with': 20,
 'if': 21,
 'that': 22,
 'is': 23,
 'okay': 24,
 'you.': 25,
 'not': 26,
 'part.': 27,
 'please.': 28,
 "you're": 29,
 'asking': 30,
 'me': 31,
 'out.': 32,
 'so': 33,
 'cute.': 34,
 "what's": 35,
 'your': 36,
 'name': 37,
 'again?': 38,
 'no,': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 '--': 43,
 "didn't": 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 '---': 48,
 'thing': 49,
 'is,': 50,
 'am': 51,
 'at': 52,
 'mercy': 53,
 'of': 54,
 'particularly': 55,
 'breed': 56,
 'loser.': 57,
 'sister.': 58,
 "can't": 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does.': 63,
 'why?': 64,
 'mystery.': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school,': 74,
 'then': 

In [26]:
answerswords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'and': 4,
 'andrew': 5,
 'are': 6,
 'having': 7,
 'an': 8,
 'incredibly': 9,
 'public': 10,
 'up': 11,
 'on': 12,
 'the': 13,
 'again.': 14,
 'well,': 15,
 'i': 16,
 'thought': 17,
 "we'd": 18,
 'start': 19,
 'with': 20,
 'if': 21,
 'that': 22,
 'is': 23,
 'okay': 24,
 'you.': 25,
 'not': 26,
 'part.': 27,
 'please.': 28,
 "you're": 29,
 'asking': 30,
 'me': 31,
 'out.': 32,
 'so': 33,
 'cute.': 34,
 "what's": 35,
 'your': 36,
 'name': 37,
 'again?': 38,
 'no,': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 '--': 43,
 "didn't": 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 '---': 48,
 'thing': 49,
 'is,': 50,
 'am': 51,
 'at': 52,
 'mercy': 53,
 'of': 54,
 'particularly': 55,
 'breed': 56,
 'loser.': 57,
 'sister.': 58,
 "can't": 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does.': 63,
 'why?': 64,
 'mystery.': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school,': 74,
 'then': 

In [27]:
len(questionswords2int)

11445

In [28]:
#질문, 답변 Dictionary에 중요한 토큰 4개를 입력(적용)함. 키와 값은 다음과 같음...

# EOS - "end of string 
# UNK - "unknown token" 
# PAD - your GPU (or CPU at worst) processes your training data in batches
# SOS - "Start of String"
# OUT - "filter out"

tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [29]:
questionswords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'and': 4,
 'andrew': 5,
 'are': 6,
 'having': 7,
 'an': 8,
 'incredibly': 9,
 'public': 10,
 'up': 11,
 'on': 12,
 'the': 13,
 'again.': 14,
 'well,': 15,
 'i': 16,
 'thought': 17,
 "we'd": 18,
 'start': 19,
 'with': 20,
 'if': 21,
 'that': 22,
 'is': 23,
 'okay': 24,
 'you.': 25,
 'not': 26,
 'part.': 27,
 'please.': 28,
 "you're": 29,
 'asking': 30,
 'me': 31,
 'out.': 32,
 'so': 33,
 'cute.': 34,
 "what's": 35,
 'your': 36,
 'name': 37,
 'again?': 38,
 'no,': 39,
 "it's": 40,
 'my': 41,
 'fault': 42,
 '--': 43,
 "didn't": 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 '---': 48,
 'thing': 49,
 'is,': 50,
 'am': 51,
 'at': 52,
 'mercy': 53,
 'of': 54,
 'particularly': 55,
 'breed': 56,
 'loser.': 57,
 'sister.': 58,
 "can't": 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does.': 63,
 'why?': 64,
 'mystery.': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school,': 74,
 'then': 

In [30]:
#making inverse dictionary of the answers2int dic  이렇게해야 숫자로 변환한 단어를 처리가능
answersints2word = {w_i: w for w, w_i in answerswords2int.items()} #딕셔너리 순서바꿈
answersints2word

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'and',
 5: 'andrew',
 6: 'are',
 7: 'having',
 8: 'an',
 9: 'incredibly',
 10: 'public',
 11: 'up',
 12: 'on',
 13: 'the',
 14: 'again.',
 15: 'well,',
 16: 'i',
 17: 'thought',
 18: "we'd",
 19: 'start',
 20: 'with',
 21: 'if',
 22: 'that',
 23: 'is',
 24: 'okay',
 25: 'you.',
 26: 'not',
 27: 'part.',
 28: 'please.',
 29: "you're",
 30: 'asking',
 31: 'me',
 32: 'out.',
 33: 'so',
 34: 'cute.',
 35: "what's",
 36: 'your',
 37: 'name',
 38: 'again?',
 39: 'no,',
 40: "it's",
 41: 'my',
 42: 'fault',
 43: '--',
 44: "didn't",
 45: 'have',
 46: 'a',
 47: 'proper',
 48: '---',
 49: 'thing',
 50: 'is,',
 51: 'am',
 52: 'at',
 53: 'mercy',
 54: 'of',
 55: 'particularly',
 56: 'breed',
 57: 'loser.',
 58: 'sister.',
 59: "can't",
 60: 'date',
 61: 'until',
 62: 'she',
 63: 'does.',
 64: 'why?',
 65: 'mystery.',
 66: 'used',
 67: 'to',
 68: 'be',
 69: 'really',
 70: 'popular',
 71: 'when',
 72: 'started',
 73: 'high',
 74: 'school,',
 75: 'the

In [31]:
# EOS 토큰을 모든 답변 뒤에 붙이기. 
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>' 

In [32]:
clean_answers[:5] #모든 답변 문장 뒤에 문장이 끝났다는 토큰 적용 완료!

["well, i thought we'd start with pronunciation, if that is okay with you. <EOS>",
 'not the hacking and gagging and spitting part.  please. <EOS>',
 "okay... then how 'bout we try out some french cuisine.  saturday?  night? <EOS>",
 'forget it. <EOS>',
 'cameron. <EOS>']

In [33]:
#이제 모든 문장을 숫자로 변환하자
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
    
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in question.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [52]:
questions_into_int[:1] #모든 문장이 숫자로 변경됨

[[0,
  1,
  2,
  3,
  11448,
  11448,
  11448,
  4,
  5,
  11448,
  6,
  7,
  8,
  9,
  11448,
  10,
  11448,
  11,
  12,
  13,
  11448,
  14]]

In [35]:
answers_into_int[:2] #답변도 모두 숫자로 변경됨

[[4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448],
 [4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448]]

In [36]:
#질문의 길이별로 질문, 답변 재구성하기
sorted_clean_questions = []
sorted_clean_answers =[]
for length in range(1, 25 + 1): #25개의 단어 길이로 반복
    for i in enumerate(questions_into_int): # 숫자로 바뀐 문장의 숫자(단어)를 하나씩 꺼내서
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [37]:
sorted_clean_questions[:4] # 단어의 길이대로 정렬됨.. 확인해보면 알게됨 ㅎㅎㅎ 지금은 4개만 확인

[[11448], [64], [125], [157]]

In [38]:
sorted_clean_answers[:5]

[[4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448],
 [4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448],
 [4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448],
 [4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448],
 [4,
  16,
  2086,
  1099,
  126,
  132,
  26,
  112,
  5086,
  328,
  68,
  6976,
  93,
  36,
  1761,
  7740,
  176,
  45,
  36,
  11448,
  11448]]